#**Zajęcia nr 4**
## Windows Factory Problem oraz dodawanie elementów interaktywnych
Na zajęciach rozpatrywano problem optymalizacji produkcji występujący w fabrykach współpracujących zz pewną firmą zajmującą się sprzedażą okien. Nowością podczas rozwiązywania problemu była możliwość dodania elementów interaktywnych takich jak paski regulacji określonych parametrów, a także tabela zawierająca:
- shadow price - czyli parametr pokazujący "wagę" ograniczenia
- slack - czyli ile zapasu mamy w manewrowaniu parametrem

Daje to możliwość eksploracji problemu oraz "gdybania" przez co można dojść do nowych rozwiązań oraz ciekawych wniosków. 

##**Zadanie do rozwiązania**

---

Pewna firma sprzedająca dwa rodzaje okien: aluminiowe oraz drewniane współpracuje z 3 fabrykami, gdzie produkowane są owe okna. Każda z fabryk posiada limit czasu, jaki może przeznaczyć na produkcję dla firmy oraz limit technologiczny co do produkcji konkretnego typu okien. Ograniczenia przedstawione zostały w poniższej tabeli:

| Fabryka | Ilość partii okien aluminiowych  produkowanych na godzinę | Ilość partii okien drewnianych produkowanych na godzinę | Dostępność w godzinach na tydzień |
|---------|-----------------------------------------------------------|---------------------------------------------------------|-----------------------------------|
| F1      | 1                                                         | 0                                                       | 4                                 |
| F2      | 0                                                         | 2                                                       | 12                                |
| F3      | 3                                                         | 2                                                       | 18                                |

Ponadto, firma sprzedaje partię okien drewnianych za 5000 dolarów,  a partię okien aluminiowych za 3000 dolarów.

Problem przed jakim stanęła firma jest następujący: Zmaksymalizowanie wartości wyprodukowanych okien przy jednoczesnym zachowaniu ograniczeń produkcyjnych fabryk

##**Rozwiązanie**

---
Najpierw zaimportowano narzędzie PULP wraz z future, ipywidgets, aby rozwiązanie problemu stało się interaktywne

In [10]:
!pip install pulp
import pulp
print(pulp.__version__)
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

2.3.1


In [11]:
from pulp import *

Zdefiniowano parametry suwaków umożliwiające zmianę niektórych wartości np. dostępności każdej z fabryk

In [12]:


style = {'description_width': 'initial'}

aluminum_price_slider = FloatSlider(min=0,max = 10, value= 3, description="aluminum_price_slider", style= style )
wooden_price_slider = FloatSlider(min=0,max = 10,value= 5, description="wooden_price_slider", style= style )
f1_eff_alu_slider = FloatSlider(min=0,max = 5,value= 1, description="f1_efficiency_aluminum", style= style )
f2_eff_alu_slider = FloatSlider(min=0,max = 5,value= 0, description="f2_efficiency_aluminum", style= style )
f3_eff_alu_slider = FloatSlider(min=0,max = 5,value= 3, description="f3_efficiency_aluminum", style= style )
f1_eff_woo_slider = FloatSlider(min=0,max = 5,value= 0, description="f1_efficiency_wooden", style= style )
f2_eff_woo_slider = FloatSlider(min=0,max = 5,value= 2, description="f1_efficiency_wooden", style= style )
f3_eff_woo_slider = FloatSlider(min=0,max = 5,value= 2, description="f1_efficiency_wooden", style= style )
f1_acc_slider = FloatSlider(min=0,max = 20,value= 4, description="f1_accessibility", style= style )
f2_acc_slider = FloatSlider(min=0,max = 20,value= 12, description="f2_accessibility", style= style )
f3_acc_slider = FloatSlider(min=0,max = 20,value= 18, description="f3_accessibility", style= style )


Zdefiniowano klasę Windows, w której odbył się cały proces optymalizacji przy pomocy programowania liniowego. Na koniec użyto funkcji Interact tworzącej interfejs użytkownika dla problemu. 

In [13]:
import pandas as pd
def Windows(aluminum_price = 3, wooden_price = 5, 
            f1_eff_alu = 1, f1_eff_woo = 0,
            f2_eff_alu = 0, f2_eff_woo = 2,
            f3_eff_alu = 3, f3_eff_woo = 2,
            f1_acc = 4, f2_acc = 12, f3_acc = 18):
    prob = LpProblem("WindowProblemwithsliders", LpMaximize)

    x1 = LpVariable("Aluminum", 0, None, LpInteger)
    x2 = LpVariable("Wooden", 0, None, LpInteger)
    
    prob += aluminum_price*x1+wooden_price*x2, "The greatest cost"

    #zmiana w formule zadania majaca na celu dostepnosc podgladu shadow price
    
    prob += x1*f1_eff_alu + x2*f1_eff_woo <= f1_acc, "f1_accessibility"
    prob += x1*f2_eff_alu + x2*f2_eff_woo <= f2_acc, "f2_accessibility"
    prob += x1*f3_eff_alu + x2*f3_eff_woo <= f3_acc, "f3_accessibility"
  
    prob.writeLP("WindowProblemSlider.lp")
    
    prob.solve()
    
    print("Status:", LpStatus[prob.status])
    
    for v in prob.variables():
        print(v.name, "=", v.varValue)
        
    print("Total income: ", value(prob.objective), "k $")
    
    shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
    print("*"*10+"Shadow Prices"+"*"*10)
    print(pd.DataFrame(shadows))
    
interact(Windows, aluminum_price = aluminum_price_slider, 
         wooden_price =  wooden_price_slider,
         f1_eff_alu = f1_eff_alu_slider,
         f1_eff_woo = f1_eff_woo_slider,
         f2_eff_alu = f2_eff_alu_slider,
         f2_eff_woo = f2_eff_woo_slider,
         f3_eff_alu = f3_eff_alu_slider,
         f3_eff_woo = f3_eff_woo_slider,
         f1_acc = f1_acc_slider,
         f2_acc = f2_acc_slider,
         f3_acc = f3_acc_slider)

interactive(children=(FloatSlider(value=3.0, description='aluminum_price_slider', max=10.0, style=SliderStyle(…

<function __main__.Windows>